In [16]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn
import pymorphy2
from pymorphy2 import MorphAnalyzer
import string
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import matplotlib_inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from gensim.models import Word2Vec
warnings.filterwarnings('ignore')
import gensim.downloader
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
nltk.download('stopwords')
nltk.download('punkt')
rus = "russian"
sw = stopwords.words(rus)
morph = MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language=rus)
vectorizer = CountVectorizer()


#ф-я для вывода текстовых данных в удобном формате
def print_table(array):
    table_width = 5
    for i in range(0, len(array), table_width):
        row = array[i:i + table_width]
        formatted_row = " ".join(["{:<10}".format(str(elem)) for elem in row])
        print(formatted_row)
#ф-я для удаления символов пунктуации
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in
    string.punctuation])
#ф-я для удаления чисел
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])
#ф-я для удаления последовательностей
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text
#ф-я для удаления не буквенных символов
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text)
#ф-я токенизации с использованием Snowball

#ф-я токенизации с помощью PyMorphy2
def tokenize_morphy(text):
    sw = stopwords.words(rus)
    # добавил слово "это", "мочь", "свой", "который", "решить", "сказать"
    sw += ["это", "мочь", "свой", "который", "решить", "сказать"]
    text = remove_notalpha(text)
    text = remove_patterns(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = text.lower()
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw:
                all_words.append("")
            else:
                all_words.append(morph.normal_forms(w)[0])
    if len(all_words) > 2:
        return ' '.join(all_words)
    else:
        return None


#Загрузка русского датасета и объедниение
train_path = 'C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\train.jsonl'
val_path = 'C:\\course_work\\feature_extraction_NLP\\data\\test_ds1\\val.jsonl'
#train_path = '/content/drive/MyDrive/course_work/data/test_ds1/train.jsonl'
#val_path = '/content/drive/MyDrive/course_work/data/test_ds1/val.jsonl'
train_data = pd.read_json(train_path, lines=True)
val_data = pd.read_json(val_path, lines=True)
df = pd.concat([train_data, val_data])
df = df.drop(['label', 'hypothesis', 'verb','negation'
,'idx','no_negation'], axis=1)
df.dropna()
df.head()
df['genre'] = df['genre'].replace('kp', 'interfax')

sw = stopwords.words(rus)
sw += ["это", "мочь", "свой", "который", "решить", "сказать"]

Y = pd.DataFrame({'genre':[str(genre) for genre in df['genre']]})
df_morphy = pd.DataFrame({'text':[tokenize_morphy(sample)
for sample in df['premise']]})
X2 = df_morphy.dropna()
batch2 = X2.join(Y, how = 'right')
batch2 = batch2.dropna()

def score_report(y_test, pred):
    acc =       accuracy_score(y_test, pred)
    recall =    recall_score(y_test, pred, average="binary", pos_label="fiction")
    precision = precision_score(y_test, pred, average="binary", pos_label="fiction")
    f1score =  f1_score(y_test, pred, average="binary", pos_label="fiction")
    scores = pd.DataFrame({"Accuracy" : [acc], "Recall" : [recall], "Precision" : [precision], "f1score" : [f1score]})
    return scores




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [18]:
tokenized = batch2['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)

In [19]:
#%pip install torch

In [20]:

import torch
from tqdm import tqdm

batch_size = 16 
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
accumulation_steps = 4 
last_hidden_states = []
for i in tqdm(range(0, len(input_ids), batch_size), desc="Processing batches"):
    batch_input_ids = input_ids[i:i+batch_size]
    batch_attention_mask = attention_mask[i:i+batch_size]
    
    with torch.no_grad():
        last_hidden_states_batch = model(batch_input_ids, attention_mask=batch_attention_mask)
    
    last_hidden_states.append(last_hidden_states_batch)
    
    torch.cuda.empty_cache()

Processing batches: 100%|██████████| 69/69 [10:35<00:00,  9.22s/it]


In [22]:
last_hidden_states

[BaseModelOutput(last_hidden_state=tensor([[[-1.9732e-01,  1.4555e-01, -1.5280e-01,  ..., -2.3614e-01,
            7.5128e-01,  6.1661e-01],
          [-6.7429e-01,  1.5450e-01,  4.3933e-01,  ...,  2.9107e-01,
            1.0756e+00,  2.9994e-01],
          [-6.5679e-01, -9.2856e-01,  2.3389e-01,  ..., -1.7869e-01,
            3.8630e-01,  2.3823e-01],
          ...,
          [ 2.7485e-02, -2.0825e-01,  2.2446e-01,  ..., -1.1231e-01,
            4.9714e-01,  8.7460e-02],
          [ 9.3034e-02, -1.7030e-01,  1.8968e-01,  ..., -1.2054e-01,
            4.8782e-01,  3.7737e-01],
          [ 1.2543e-01, -2.7040e-01,  1.5603e-01,  ..., -1.7182e-02,
            6.3556e-01,  3.7317e-01]],
 
         [[-2.2584e-01,  1.1905e-01, -1.5882e-01,  ..., -2.9262e-01,
            6.1920e-01,  5.1746e-01],
          [-3.7023e-01,  1.3369e-01,  1.9956e-01,  ...,  6.1893e-03,
            5.4699e-01,  6.8173e-01],
          [-6.0592e-01, -1.7610e-01,  1.3046e-01,  ..., -3.1732e-01,
            9.9968e-01,

In [41]:
import numpy
l = [last_hidden_states[i][0][:,0,:].numpy() for i in range(69)]
l2 = []
for i in range(69):
    for j in range(len(l[i])):
        l2.append(l[i][j])

features = numpy.array(l2)
labels = batch2['genre']

In [42]:
features.shape

(1096, 768)

In [ ]:
labels.shape

(1096,)

In [43]:

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)

In [44]:
bert_token = pd.DataFrame(features, columns = [f'col{x}' for x in range(len(features[0]))])
bert_token.assign(label = labels)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col759,col760,col761,col762,col763,col764,col765,col766,col767,label
0,-0.197318,0.145548,-0.152800,-0.350353,-0.050346,-0.013122,0.286338,0.079355,-0.228672,-0.065834,...,-0.574910,0.343822,-0.175194,0.425219,-0.703991,-0.000673,-0.236139,0.751281,0.616612,interfax
1,-0.225843,0.119055,-0.158819,-0.347235,-0.122859,0.057133,0.286214,0.158213,-0.222860,-0.170866,...,-0.529721,0.331591,-0.179737,0.389557,-0.633088,0.005149,-0.292623,0.619201,0.517461,fiction
2,-0.200367,0.199965,-0.183570,-0.345064,-0.174476,0.014465,0.296963,0.117709,-0.201017,-0.178382,...,-0.619925,0.361125,-0.142961,0.334128,-0.632685,0.018041,-0.315487,0.702101,0.515160,fiction
3,-0.241902,0.079609,-0.188643,-0.347283,-0.041564,0.126903,0.309099,0.257407,-0.186384,-0.271733,...,-0.442407,0.221565,-0.153047,0.377826,-0.559655,0.012197,-0.246348,0.473959,0.409792,interfax
4,-0.211857,0.176365,-0.177572,-0.338686,-0.045318,-0.005752,0.265812,0.145152,-0.264204,-0.067932,...,-0.560684,0.343589,-0.171606,0.402748,-0.682718,-0.020298,-0.276541,0.709944,0.546235,fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,-0.156812,0.148506,-0.125250,-0.315194,-0.052328,0.001617,0.287678,0.077609,-0.221625,-0.063419,...,-0.558874,0.330174,-0.215812,0.352460,-0.694556,-0.008057,-0.235899,0.715721,0.569725,fiction
1092,-0.201947,0.063755,-0.150137,-0.383300,-0.147376,0.057245,0.308365,0.209313,-0.224729,-0.211025,...,-0.521601,0.268973,-0.120436,0.307058,-0.558960,0.046995,-0.308155,0.491679,0.522037,fiction
1093,-0.182591,0.104261,-0.153534,-0.345847,-0.057654,0.006084,0.251989,0.122018,-0.280116,-0.086681,...,-0.562777,0.338176,-0.218451,0.367617,-0.697620,0.019530,-0.276259,0.671830,0.539817,interfax
1094,-0.190726,0.218665,-0.072998,-0.359409,-0.123375,-0.028747,0.253239,-0.011296,-0.232212,-0.117684,...,-0.527794,0.341988,-0.173298,0.444171,-0.678531,-0.056283,-0.293604,0.797236,0.653422,interfax


In [45]:
bert_token.to_csv('bert_vectors.csv', index=False)

In [49]:
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [50]:
svc = SVC()
svc.fit(train_features, train_labels)
pred = svc.predict(test_features)
score_report(test_labels, pred)

,Accuracy,Recall,Precision,f1score
0,0.576642,0.986301,0.55814,0.712871


In [47]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
test_pred = lr_clf.predict(test_features)

score_report(test_labels, test_pred)

,Accuracy,Recall,Precision,f1score
0,0.733577,0.828767,0.715976,0.768254


In [10]:
#(3 + x^2 + x^5)^19
s = 0
import math as m
ntf = m.factorial(19)
for m1 in range(20):
    for m2 in range(20):
        for m3 in range(20):
            if (m1 + m2 + m3) == 19 and (2*m2 + 5*m3) == 30:
                print(m1, m2, m3)
                s += ntf / (m.factorial(m1)*m.factorial(m2)*m.factorial(m3)) * m.pow(3, m1)
                print(ntf / (m.factorial(m1)*m.factorial(m2)*m.factorial(m3)) * m.pow(3, m1))
print('s= ', s)

4 15 0
313956.0
7 10 2
7273104696.0
10 5 4
687308393772.0
13 0 6
43257171636.0
s=  737838984060.0


In [16]:
#(1 + sqrt(8))^100
mx = -1
p = 0
sq8 = m.sqrt(8)
for i in range(101):
    if m.comb(100, 1)*((sq8)**i) > mx:
        mx = m.comb(100, i)*((sq8)**i)
        p = i
        print(100 - i, i, m.comb(100, i)*((sq8)**i))
print(mx, p)
    

100 0 1.0
99 1 282.842712474619
98 2 39600.00000000001
94 6 610330828800.0002
78 22 6.298197496784534e+31
34 66 3.680733991743417e+56
3.680733991743417e+56 66
